In [70]:
import pandas as pd
import seaborn as sb
import csv
from sympy import FiniteSet

import pysam
import os
import math
import random
import numpy as np

from pathlib import Path
from random import seed
from random import randint
from functools import reduce
#from Numeric import *
#from LinearAlgebra import singular_value_decomposition
from operator import add as concat
#from more_itertools import powerset
from itertools import combinations
from itertools import chain, combinations


In [56]:
os.makedirs('output-tda/db-bowtie')

## Crear directorios por cada archivo de TP's para usarlos con bowtie:

### Ruta a los Reads:
/home/shaday/GIT/TDA-Metaganomas/Reads_DWGSIM

In [163]:
folder = "reads" 

#ruta_reads = '/home/shaday/GIT/TDA-Metaganomas/ReadsSimulados/Salmonella/'+folder
ruta_reads = '/home/shaday/GIT/TDA-Metaganomas/Reads_DWGSIM'
ruta_genomas = '/home/shaday/GIT/TDA-Metaganomas/genomes/salmonella/'  ##Se queda igual
ruta_db = '/home/shaday/GIT/TDA-Metaganomas/output-tda/db-bowtie/'     ##Se queda igual
ruta_ids_db = "/home/shaday/GIT/TDA-Metaganomas/list-DBsalmonella.txt"  
ruta_output_bowtie = "/home/shaday/GIT/TDA-Metaganomas/output-tda/"
 
ruta_csv_files = ""

In [176]:
contenido = list(os.listdir(ruta_genomas))
contenido = [na for na in contenido if "fasta" in na]
contenido.sort()
#contenido_names = [contenido[i][0:8] for i in range(len(contenido)) if i%3==0]
contenido_names = [contenido[i][0:8] for i in range(0,12) ]
contenido_names = [nam for nam in contenido_names if "." not in nam]
contenido_names

['CP019116',
 'CP019403',
 'CP019404',
 'CP019405',
 'CP019406',
 'CP019407',
 'CP019408',
 'CP019409',
 'CP019410',
 'CP019411',
 'CP019412',
 'CP019413']

In [177]:
for folder1 in contenido_names:
    os.makedirs('output-tda/'+folder1)

    os.makedirs('output-tda/'+folder1+'/out-bowtie')
    os.makedirs('output-tda/'+folder1+'/bam')
    os.makedirs('output-tda/'+folder1+'/sorted')
    os.makedirs('output-tda/'+folder1+'/tablas')
    os.makedirs('output-tda/'+folder1+'/SyReads')
    os.makedirs('output-tda/'+folder1+'/sam_extract')
    os.makedirs('output-tda/'+folder1+'/depth')
    

## Correr bowtie

In [183]:
%%bash -s "$ruta_reads" "$ruta_db" "$ruta_ids_db"
source /opt/anaconda3/etc/profile.d/conda.sh
conda activate metagenomics
ls $1/*read1.fastq.gz | head -12 | while read lectura
do
    id=$(basename ${lectura} .bwa.read1.fastq.gz)
    cat $3 | head -12 | while read line
    do
        bowtie2 -x $2$line -1 $1/${id}.bwa.read1.fastq.gz -2 $1/${id}.bwa.read2.fastq.gz --no-unal -p 12 -S output-tda/$id/out-bowtie/TP_${id}_to_${line}.sam 
        #echo "bowtie2 -x $2$line -1 $1/${id}_R1.fastq -2 $1/${id}_R2.fastq --no-unal -p 12 -S output-tda/$id/out-bowtie/TP_${id}_to_${line}.sam" 
    done
done;

10000 reads; of these:
  10000 (100.00%) were paired; of these:
    1 (0.01%) aligned concordantly 0 times
    9804 (98.04%) aligned concordantly exactly 1 time
    195 (1.95%) aligned concordantly >1 times
    ----
    1 pairs aligned concordantly 0 times; of these:
      1 (100.00%) aligned discordantly 1 time
    ----
    0 pairs aligned 0 times concordantly or discordantly; of these:
      0 mates make up the pairs; of these:
        0 (0.00%) aligned 0 times
        0 (0.00%) aligned exactly 1 time
        0 (0.00%) aligned >1 times
100.00% overall alignment rate
10000 reads; of these:
  10000 (100.00%) were paired; of these:
    1308 (13.08%) aligned concordantly 0 times
    8568 (85.68%) aligned concordantly exactly 1 time
    124 (1.24%) aligned concordantly >1 times
    ----
    1308 pairs aligned concordantly 0 times; of these:
      234 (17.89%) aligned discordantly 1 time
    ----
    1074 pairs aligned 0 times concordantly or discordantly; of these:
      2148 mates make u

In [ ]:
#ruta_tablas = '/output-tda/'+organism+'/tablas/'

In [184]:
%%bash -s "$ruta_reads" "$ruta_db" "$ruta_ids_db"
#ls $1/*_R1.fastq | while read lectura
ls $1/*read1.fastq.gz | head -12 | while read lectura
do
    #id=$(basename ${lectura} _R1.fastq)
    id=$(basename ${lectura} .bwa.read1.fastq.gz)
    for infile in output-tda/$id/out-bowtie/*.sam
        do 
            base=$(basename ${infile} .sam)
            samtools view ${infile} -o output-tda/$id/bam/${base}.bam
            samtools sort ${infile} -o output-tda/$id/sorted/${base}_sorted.sam
        done
done

In [185]:
%%bash -s "$ruta_reads" "$ruta_db" "$ruta_ids_db"
#ls $1/*_R1.fastq | while read lectura
ls $1/*read1.fastq.gz | head -12 | while read lectura
do
    #id=$(basename ${lectura} _R1.fastq)
    id=$(basename ${lectura} .bwa.read1.fastq.gz)
    for infile in output-tda/$id/bam/*.bam
        do 
            base=$(basename ${infile} .bam)
            samtools sort ${infile} -o output-tda/$id/sorted/${base}_sorted.bam
            samtools view ${infile} > output-tda/$id/bam/${base}.txt
        done
    ls output-tda/$id/bam/*.txt | while read line
        do
            base=$(basename ${line} .txt)
            cut -f 1-6 $line > output-tda/$id/tablas/${base}_extrac.txt
        done
done

In [92]:
#ruta_tablas = '/home/cesar/TDA/TDA-Metaganomas/output-tda/'+folder1+'/tablas/'

#Elegir donde se desea que se guarde la salida de bowtie:
path_output_bowtie = "/home/cesar/TDA/TDA-Metaganomas/test_salmonella/output-tda/"  

In [112]:
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f

def construct_csv(path, folder):
    ruta_tablas = path + folder + "/tablas/"
    contenido1 = list(listdir_nohidden(ruta_tablas))
    #genomes = []
    df=[]
    for i in contenido1:
        dftt = pd.read_csv(ruta_tablas + i, sep='\t', header=None) #importar las tablas para cada genoma
        dftt['Organism'] = i[15:23] #agregar el nombre del organismo
        df.append(dftt) #agregar al dataframe
        #genomes.append(i)
    
    df=pd.concat(df)
    #Nombrando las columnas del data frame
    df.columns= ['Qname','Flag', 'Rname', 'Pos','MapQ','CIGAR','Organism'] 
    df.to_csv('/home/cesar/TDA/TDA-Metaganomas/CSV/salmonella/1TP/6org/ROM_1TP_'+ folder + '.csv')
    

In [152]:
name='TP_CP019116_to_CP022503_extrac'
'CP022503' in name.split('_')


'TP_CP019116_to_CP022503_extrac'

In [158]:
def construct_csv2(path, folder, lista_nombres):
    ruta_tablas = path + folder + "/tablas/"
    contenido1 = list(listdir_nohidden(ruta_tablas))
    #genomes = []
    df=[]
    for i in contenido1:
        if (i.split("_")[3] in lista_nombres): 
            dftt = pd.read_csv(ruta_tablas + i, sep='\t', header=None) #importar las tablas para cada genoma
            dftt['Organism'] = i[15:23] #agregar el nombre del organismo
            df.append(dftt) #agregar al dataframe
            #genomes.append(i)
    
    df=pd.concat(df)
    df.columns= ['Qname','Flag', 'Rname', 'Pos','MapQ','CIGAR','Organism'] #cambiar nombre a las columnas del dataframe
    df.to_csv('/home/cesar/TDA/TDA-Metaganomas/CSV/salmonella/1TP/6org/ROM_1TP_'+ folder + '.csv')


In [187]:
lista_org = contenido_names[0:12]
lista_org

['CP019116',
 'CP019403',
 'CP019404',
 'CP019405',
 'CP019406',
 'CP019407',
 'CP019408',
 'CP019409',
 'CP019410',
 'CP019411',
 'CP019412',
 'CP019413']

In [188]:

#for fold in contenido_names[0:6]:
for fold in lista_org:
    construct_csv2("/home/cesar/TDA/TDA-Metaganomas/test_salmonella/output-tda/", fold, lista_org)

In [199]:
df1 = pd.read_csv("../CSV/salmonella/1TP/6org/ROM_1TP_CP019116.csv")

In [200]:
df1["Organism"].value_counts()

CP019116    20000
CP019412    18610
CP019411    18598
CP019413    18309
CP019404    18226
CP019408    18201
CP019409    18152
CP019405    18100
CP019403    18029
CP019410    17983
CP019406    17982
CP019407    17891
Name: Organism, dtype: int64

In [204]:
df2 = pd.read_csv("../CSV/salmonella/1TP/6org/ROM_1TP_CP019413.csv")
df2["Organism"].value_counts()

CP019413    20000
CP019407    18117
CP019405    17878
CP019408    17725
CP019403    17717
CP019404    17710
CP019406    17606
CP019116    17184
CP019410    16895
CP019412    16871
CP019409    16870
CP019411    16706
Name: Organism, dtype: int64

In [206]:
df2

,Unnamed: 0,Qname,Flag,Rname,Pos,MapQ,CIGAR,Organism
0,0,CP019413.1_1477622_1477972_0_1_0_0_0:0:0_1:0:0_0,83,CP019403.1,1846140,42,150M,CP019403
1,1,CP019413.1_1477622_1477972_0_1_0_0_0:0:0_1:0:0_0,163,CP019403.1,1845790,42,150M,CP019403
2,2,CP019413.1_4553553_4553203_1_0_0_0_0:0:0_0:0:0_1,83,CP019403.1,4443467,21,150M,CP019403
3,3,CP019413.1_4553553_4553203_1_0_0_0_0:0:0_0:0:0_1,163,CP019403.1,4443117,21,150M,CP019403
4,4,CP019413.1_1250739_1250389_1_0_0_0_1:0:0_1:0:0_2,99,CP019403.1,2019001,40,93M1I56M,CP019403
...,...,...,...,...,...,...,...,...
211274,16701,CP019413.1_3820826_3821176_0_1_0_0_1:0:0_6:0:0...,147,CP019411.1,3601358,40,150M,CP019411
211275,16702,CP019413.1_4683809_4683459_1_0_0_0_0:0:0_1:0:0...,83,CP019411.1,4446494,42,150M,CP019411
211276,16703,CP019413.1_4683809_4683459_1_0_0_0_0:0:0_1:0:0...,163,CP019411.1,4446144,42,150M,CP019411
211277,16704,CP019413.1_4323029_4322679_1_0_0_0_0:0:0_0:0:1...,83,CP019411.1,4078864,42,150M,CP019411
